# Quantum State Tomography

In [1]:
import qiskit
import qiskit_experiments as qe
tomo = qe.tomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = tomo.StateTomographyExperiment(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100)
qstresult = qstdata1.analysis_result(-1)

print('FIT RESULT')
for key, val in qstresult.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 0.48046875+0.j        ,  0.00960286-0.01546224j,
                -0.02132161+0.00569661j,  0.00488281-0.45410156j],
               [ 0.00960286+0.01546224j,  0.03125   +0.j        ,
                 0.00683594+0.00488281j,  0.01676432+0.0008138j ],
               [-0.02132161-0.00569661j,  0.00683594-0.00488281j,
                 0.02734375+0.j        ,  0.00374349+0.00211589j],
               [ 0.00488281+0.45410156j,  0.01676432-0.0008138j ,
                 0.00374349-0.00211589j,  0.4609375 +0.j        ]],
              dims=(2, 2))
fitter: linear_inversion
fitter_time: 0.002797842025756836
state_eigvals: [0.92586773 0.04276179 0.02850229 0.00286819]
state_trace: 0.9999999999999999
value: 0.9248046874999998
value_label: state_fidelity
success: True


### Tomography Results

The main results for tomography are the fitted state, which is stored in the `"state"` key as a `DensityMatrix` object:

In [3]:
print(qstresult["state"])

DensityMatrix([[ 0.48046875+0.j        ,  0.00960286-0.01546224j,
                -0.02132161+0.00569661j,  0.00488281-0.45410156j],
               [ 0.00960286+0.01546224j,  0.03125   +0.j        ,
                 0.00683594+0.00488281j,  0.01676432+0.0008138j ],
               [-0.02132161-0.00569661j,  0.00683594-0.00488281j,
                 0.02734375+0.j        ,  0.00374349+0.00211589j],
               [ 0.00488281+0.45410156j,  0.01676432-0.0008138j ,
                 0.00374349-0.00211589j,  0.4609375 +0.j        ]],
              dims=(2, 2))


Additional data stored in the tomography result including:
- `"value"`: The `state_fidelity` with the ideal target state
- `"state_trace"`: The trace of the fitted state (sum of state eigenvalues)
- `"state_eigvals"`: the eigenvalues of the fitted state

In [4]:
print("State fidelity =", qstresult["value"])
print("State eigenvalues =", qstresult["state_eigvals"])
print("State trace =", qstresult["state_trace"])

State fidelity = 0.9248046874999998
State eigenvalues = [0.92586773 0.04276179 0.02850229 0.00286819]
State trace = 0.9999999999999999


#### Additional result metadata

If the fitted returned a non-positive hat was rescaled to be PSD there will be additional fields in the metadata containing the raw (unscaled) eigenvalues:

- `"raw_eigenvalues"`: the eigenvalues of the state before rescaling
- `"rescaled_psd:` : True if PSD rescaling was performed.

If trace rescaling was performed because the input state was not trace 1, there will also be extra fields:
- `"raw_trace"`: the sum of the raw eigenvalues before rescaling
- `"rescaled_trace"`: True if trace rescaling was performed

To see the effect of rescaling we can perform a "bad" fit with very low counts

In [5]:
# QST Experiment
bad_data = qstexp1.run(backend, shots=10, seed_simulation=100)
bad_result = bad_data.analysis_result(-1)

print('FIT RESULT')
for key, val in bad_result.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 0.41889088+0.j        , -0.01901677+0.00832074j,
                -0.03035713+0.02792112j, -0.09311667-0.39729055j],
               [-0.01901677-0.00832074j,  0.05838767+0.j        ,
                -0.0229442 -0.0030119j ,  0.02379613-0.05475337j],
               [-0.03035713-0.02792112j, -0.0229442 +0.0030119j ,
                 0.01494642+0.j        , -0.02858843+0.06849366j],
               [-0.09311667+0.39729055j,  0.02379613+0.05475337j,
                -0.02858843-0.06849366j,  0.50777504+0.j        ]],
              dims=(2, 2))
fitter: linear_inversion
fitter_time: 0.0015001296997070312
rescaled_psd: True
state_eigvals: [0.88304004 0.11695996 0.         0.        ]
raw_eigvals: [ 0.97365872  0.20757865  0.03681031 -0.21804768]
state_trace: 0.9999999999999982
value: 0.8606235026683007
value_label: state_fidelity
success: True


## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [6]:
qstexp1.run_analysis(qstdata1, fitter='cvxpy_linear_lstsq')
qstresult2 = qstdata1.analysis_result(-1)

print('FIT RESULT')
for key, val in qstresult2.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 0.48047131+0.00000000e+00j,  0.00960283-1.54624197e-02j,
                -0.0213217 +5.69679365e-03j,  0.00488287-4.54106140e-01j],
               [ 0.00960283+1.54624197e-02j,  0.03124744+1.08420217e-18j,
                 0.00683582+4.88319131e-03j,  0.01676447+8.13703003e-04j],
               [-0.0213217 -5.69679365e-03j,  0.00683582-4.88319131e-03j,
                 0.02734128+1.73472348e-18j,  0.00374364+2.11608818e-03j],
               [ 0.00488287+4.54106140e-01j,  0.01676447-8.13703003e-04j,
                 0.00374364-2.11608818e-03j,  0.46093997+8.67361738e-19j]],
              dims=(2, 2))
cvxpy_solver: CVXOPT
cvxpy_status: optimal
fitter: cvxpy_linear_lstsq
fitter_time: 0.07893776893615723
rescaled_trace: True
state_eigvals: [0.92587484 0.04275964 0.0284995  0.00286602]
raw_eigvals: [0.92587372 0.04275959 0.02849947 0.00286602]
state_trace: 1.0
raw_trace: 0.999998798152777
value: 0.9248117804376139
value_label: state_fidelity
success: True


## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [7]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    tomo.StateTomographyExperiment(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = qe.composite.ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=100)
print(pardata.analysis_result(-1))


- experiment_types: ['StateTomographyExperiment', 'StateTomographyExperiment', 'StateTomographyExperiment', 'StateTomographyExperiment', 'StateTomographyExperiment']
- experiment_ids: ['fafe4878-8b66-4c9d-939c-a74e5708e9c9', '2c8e695c-b042-414d-a46b-61699e522da7', '67a92223-4b0a-4d3c-9bbc-de7a12ccd8b3', 'a235c5e5-e6b3-4307-9aae-d1644b097a7d', '818f5711-8991-4c09-b3a1-20cb727e6d95']
- experiment_qubits: [(0,), (1,), (2,), (3,), (4,)]
- success: True


View component experiment analysis results

In [8]:
for i in range(parexp.num_experiments):
    expdata = pardata.component_experiment_data(i)
    result = expdata.analysis_result(-1)
    
    print(f'\nPARALLEL EXP {i}: FIT RESULT')
    for key, val in result.items():
        print(f'{key}:', val)


PARALLEL EXP 0: FIT RESULT
state: DensityMatrix([[ 0.98046875+0.j      , -0.00195312-0.015625j],
               [-0.00195312+0.015625j,  0.01953125+0.j      ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.00022101402282714844
state_eigvals: [0.98072672 0.01927328]
state_trace: 1.0
value: 0.9804687499999997
value_label: state_fidelity
success: True

PARALLEL EXP 1: FIT RESULT
state: DensityMatrix([[0.84179687+0.j        , 0.04492188+0.34082031j],
               [0.04492188-0.34082031j, 0.15820313+0.j        ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.00011491775512695312
state_eigvals: [0.9847696 0.0152304]
state_trace: 0.9999999999999999
value: 0.9826832422357389
value_label: state_fidelity
success: True

PARALLEL EXP 2: FIT RESULT
state: DensityMatrix([[ 0.49511719+0.j        , -0.00683594+0.47265625j],
               [-0.00683594-0.47265625j,  0.50488281+0.j        ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.0001